# Planning

In [141]:
import pandas as pd
import datetime
from configparser import ConfigParser
import json, requests, datetime
from pysimplicate import Simplicate

# Pandas
pd.options.display.float_format = '{:,.1f}'.format
pd.set_option('display.max_columns', 500)
PANDAS_FILE = '../simplicate_cache/hours.pd'
full_df = pd.read_pickle(PANDAS_FILE)

df = full_df.copy().query( 'type=="normal"')
df['turnover'] = df.apply(lambda a: (a['hours']+a['corrections']) * (a['tariff'] if a['tariff'] > 0 else a['service_tariff']), axis=1)
df['turnover'] = df.apply(lambda a: a['turnover']/2 if a['project_number'] == 'TOR-3' else a['turnover'], axis=1)
df['week'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').isocalendar()[1], axis=1)
df['month'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').month, axis=1)

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

## Probeersel om een beeld te krijgen van toekomstige omzet

In [142]:
def active_projects():
    projects = [{'project': project.get('project_number',''),
                 'spent' : project['budget']['total'].get('value_spent', 0),
                 'budget' : project['budget']['total']['value_budget']
                }
                for project in sim.project( {'active':True} )]
    return projects

def todo_work_list():

    def corrections( project ):
        return df.query(f'project_number=="{project}"')['corrections'].sum()

    return pd.DataFrame([{'project': project['project'],
                         'budget': project['budget'],
                         'spent': project['spent'],
                         'todo': project['budget'] - project['spent']}
                        for project in active_projects()]).sort_values(by=['todo'], ascending=True)


todo = todo_work_list()
todo

,project,budget,spent,todo
2,TOR-3,0.0,"99,512.5","-99,512.5"
1,ACC-1,0.0,"97,227.5","-97,227.5"
13,BAM-1,0.0,"50,072.6","-50,072.6"
4,BEN-1,0.0,"38,555.6","-38,555.6"
42,MTM-1,0.0,"26,715.0","-26,715.0"
...,...,...,...,...
37,OER-1,"35,000.0","30,377.5","4,622.5"
22,THIE-20,"13,710.0","7,380.0","6,330.0"
45,HAVA-2,"25,420.0","16,173.8","9,246.2"
5,SM2021,"44,370.0","35,020.0","9,350.0"


In [143]:
def double_get(obj, first, second):
    if obj.get(first):
        return obj[first].get(second, '')
    return ''

def sales_flat(self):
    return [
        {
            'organization': s['organization']['name'],
            'subject': s['subject'],
            'employee': double_get(s, 'responsible_employee', 'name'),
            'progress_label': s['progress']['label'],
            'progress_position': s['progress']['position'],
            'status': s['status']['label'],
            'start_date': s['start_date'],
            'modified': s['modified'],
            'source': double_get(s, 'source', 'name'),
            'expected_revenue': s['expected_revenue'],
            'chance_to_score': s['chance_to_score'],
            'expected_closing_date': s.get('expected_closing_date'),
            'value': s['expected_revenue'] * s['chance_to_score'] / 100,
        }
        for s in self.sales()
        if s['status']['label'] == 'OpportunityStatus_open'
    ]

pd.DataFrame(sales_flat(sim))

,organization,subject,employee,progress_label,progress_position,status,start_date,modified,source,expected_revenue,chance_to_score,expected_closing_date,value
0,PlayFountain B.V.,Klantomgeving en dashboard Playfountain,Joost Cornelissen,On Hold,2,OpportunityStatus_open,2020-05-26,2021-02-23 12:43:17,Via partner,50000,0,None,0.0
1,Stichting JobOn,JobOn learning dashboard mbv SLIM subsidie,Joost Cornelissen,On Hold,2,OpportunityStatus_open,2020-06-08,2021-02-23 12:40:01,Bestaande klant,25000,10,2021-04-30,"2,500.0"
2,ThiemeMeulenhoff B.V.,Corporate website ThiemeMeulenhoff,Joost Cornelissen,Toekomst,1,OpportunityStatus_open,2020-06-08,2021-02-23 12:38:48,,100000,10,2021-09-03,"10,000.0"
3,Sprout Money BV,Refactoring slimbeleggen.com (frontend),Stefan Roovers,Inventarisatie,3,OpportunityStatus_open,2020-06-11,2021-02-23 12:45:41,Bestaande klant,60000,20,None,"12,000.0"
4,Sprout Money BV,Refactoring checkouts - development,Stefan Roovers,Inventarisatie,3,OpportunityStatus_open,2020-08-04,2021-02-23 12:45:55,Bestaande klant,35460,30,None,"10,638.0"
5,Sprout Money BV,Bifirst Marketing website (design en develpment),Joost Cornelissen,Inventarisatie,3,OpportunityStatus_open,2020-11-16,2021-02-10 15:49:01,Bestaande klant,25000,80,2021-03-01,"20,000.0"
6,Management Drives Netherlands B.V.,Sale nieuwe strippenkaart van 8 strippen,Gerben van Dijk,Onderhandeling,7,OpportunityStatus_open,2021-01-11,2021-01-11 13:57:54,,880,50,None,440.0
7,ThiemeMeulenhoff B.V.,Release #2 nav Design Sprint,Joost Cornelissen,Ballpark inschatting,4,OpportunityStatus_open,2021-01-26,2021-01-26 09:11:26,Bestaande klant,35000,80,2021-03-12,"28,000.0"
8,Sprout Money BV,Bitfirst marcom website en support (ongoing),Joost Cornelissen,On Hold,2,OpportunityStatus_open,2021-01-26,2021-02-26 16:40:49,Bestaande klant,15000,50,None,"7,500.0"
9,Sprout Money BV,"Copycoin Visie, CI + PPO",Joost Cornelissen,Inventarisatie,3,OpportunityStatus_open,2021-01-26,2021-02-10 16:29:49,Bestaande klant,30000,50,2021-03-01,"15,000.0"


# Verzuim

In [144]:
DATE_FORMAT = '%Y-%m-%d'
from_day = max('2021-01-01', (datetime.datetime.today() + datetime.timedelta(days=-90)).strftime(DATE_FORMAT))
verzuim = full_df.query( f'type=="absence" and day >="{from_day}"')['hours'].sum()
normal = full_df.query( f'type=="normal" and day >="{from_day}"')['hours'].sum()
percentage = 100 * verzuim / (normal+verzuim)
percentage

2.2623642872969407

# Roosterdagen

In [145]:
def tuple_of_productie_users():
    productie_teams = {'Development', 'PM', 'Service Team', 'Concept & Design', 'Testing'}
    users = sim.employee({'employment_status': 'active'})
    users = [u['name'] for u in users if set(t['name'] for t in u.get('teams', [])).intersection(productie_teams)]
    return users
productie_users = tuple_of_productie_users()
productie_users

['Gerben van Dijk',
 'Sebastian Schipper',
 'Filipe José Mariano dos Santos',
 'Geert-Jan van Mastrigt',
 'George Tsimenis',
 'Marijn Hurkens',
 'Kevin Lobine',
 'Stefan Roovers',
 'Bas Colenbrander',
 'Caspar Geerlings',
 'Jie Chin',
 'Chris Neven',
 'Fadhlur Zahri',
 'Gijs Kattenberg',
 'Jari Zwarts',
 'Jeroen Soeteman',
 'Jochem Tijhuis',
 'Jordi Hendrix',
 'Jurriaan Ruitenberg',
 'Mark Kuijer',
 'Matthijs Blankevoort',
 'Paulo Nuno da Cruz Moreno',
 'Raymond Kiekens',
 'Richard van Willegen',
 'Rik Scheffer',
 'Robin Veer',
 'Rogier Voogt',
 'Sho Stegmeijer',
 'Max van Deurzen',
 'Patricia Snel',
 'Eva Korteweg',
 'Kelly de Haan',
 'Kheira Oudejans',
 'Mandy Dorée',
 'Miek ten Brummelhuis']

In [146]:


timetable = [t for t in sim.timetable() if not t.get('end_date') and t['employee']['name'] in productie_users]
odd = {table['employee']['name']:[table['odd_week'][f'day_{i}']['hours'] for i in range(1, 6)] 
       for table in timetable}
even = {table['employee']['name']:[table['even_week'][f'day_{i}']['hours'] for i in range(1, 6)] 
       for table in timetable}

odd_tot = sum([sum(week) for week in odd.values()])
even_tot = sum([sum(week) for week in even.values()])
odd_tot, even_tot
def weektot( weekno ):
    return even_tot if int(weekno) % 2 == 0 else odd_tot

In [147]:
from hplib.dbclass import dbClass

db = dbClass.from_inifile( '../credentials.ini')
def dataframe(query):
    return 

last_day = '2021-01-01'
query = f'''select week(day,5) as weekno, ifnull(round(sum(dayhours)),0) as plannedhours from
    (select day, sum(hours) as dayhours from
        (select date(startDate) as day,
                sum(least((enddate - startDate)/10000,8)) as hours
         from planning_reservation pr
         join planning_location pl on pl.id=pr.planning_locationId
         left join project p on p.id=pr.projectId
         where startDate > "{last_day}" AND planning_typeId = '17' and (p.customerId is null or p.customerId <> 4)
         group by day) q1
    group by day) q2
group by year(day), weekno
order by day'''
table = pd.read_sql_query(query, db.db)
table['roster'] = table.apply(lambda a: even_tot if a['weekno'] % 2 == 0 else odd_tot, axis=1)
leaves = pd.DataFrame([{'day':l['start_date'].split()[0], 'week':int(datetime.datetime.strptime(l['start_date'].split()[0],DATE_FORMAT).strftime('%W')), 
                        'hours':-l['hours'], 'employee':l['employee']['name']} 
                       for l in sim.leave({'start_date':'2021-01-01'})])
leave_hours_per_week = leaves.groupby(['week']).sum(['hours'])
table['leaves'] = table.apply(lambda row: leave_hours_per_week.at[row['weekno'],'hours'] if row['weekno'] in leave_hours_per_week.index else 0, axis=1)
table['filled'] = table.apply( lambda row: int(100 * row['plannedhours'] / (row['roster']-row['leaves'])), axis=1)
table

,weekno,plannedhours,roster,leaves,filled
0,1,662.0,1306,146,57
1,2,902.0,1250,18,73
2,3,784.0,1306,65,63
3,4,884.0,1250,66,74
4,5,800.0,1306,23,62
5,6,874.0,1250,58,73
6,7,847.0,1306,8,65
7,8,953.0,1250,46,79
8,9,830.0,1306,12,64
9,10,791.0,1250,0,63


In [151]:
table[['weekno','filled']].to_dict('records')

[{'weekno': 1, 'filled': 57},
 {'weekno': 2, 'filled': 73},
 {'weekno': 3, 'filled': 63},
 {'weekno': 4, 'filled': 74},
 {'weekno': 5, 'filled': 62},
 {'weekno': 6, 'filled': 73},
 {'weekno': 7, 'filled': 65},
 {'weekno': 8, 'filled': 79},
 {'weekno': 9, 'filled': 64},
 {'weekno': 10, 'filled': 63},
 {'weekno': 11, 'filled': 48},
 {'weekno': 12, 'filled': 53},
 {'weekno': 13, 'filled': 35},
 {'weekno': 14, 'filled': 26},
 {'weekno': 15, 'filled': 17},
 {'weekno': 16, 'filled': 17},
 {'weekno': 17, 'filled': 12},
 {'weekno': 18, 'filled': 3},
 {'weekno': 19, 'filled': 0}]